In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import plotly.express as px
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
!pip install pycountry-convert

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data=pd.read_csv('../input/women-entrepreneurship-and-labor-force/Dataset3.csv',sep=';')

In [ ]:
data.head()
data.reset_index(drop=True, inplace=True)

In [ ]:
data.info()

# **ASSIGNING COUNTRIES THEIR CONTINENTS**

In [ ]:
import pycountry_convert as pc

def country_to_continent(country_name):
    country_alpha2 = pc.country_name_to_country_alpha2(country_name)
    country_continent_code = pc.country_alpha2_to_continent_code(country_alpha2)
    country_continent_name = pc.convert_continent_code_to_continent_name(country_continent_code)
    return country_continent_name

# Example
for i in data["Country"]:
    data.loc[data["Country"]==i,["Continent"]]=country_to_continent(i)
    
    
def country_to_continent(country_name):
    country_alpha3 = pc.country_name_to_country_alpha3(country_name)
    return country_alpha3
for i in data["Country"]:
    data.loc[data["Country"]==i,["iso_alpha"]]=country_to_continent(i)

In [ ]:
data.sample(10)

# Choropleth using the given data

In [ ]:
import plotly.express as px
fig = px.choropleth(data, locations="iso_alpha", color="Entrepreneurship Index", hover_name="Country", 
                    hover_data=["Women Entrepreneurship Index","Inflation rate","Female Labor Force Participation Rate","Currency","Level of development"]
                    ,height=900,width=900,range_color=[20,80])
fig.show()

# **CORRELATION OF COLUMNS**

In [ ]:
df=data.corr()
# here i have stored the data of correlation of the variables with respect to each other in a variable x

In [ ]:
fig = px.imshow(df)
fig.update_xaxes(side="top")
fig.show()

# **CONCLUSIONS:**
*  ENTRPRENEURSHIP INDEX AND WOMEN ENTREPRENEURSHIP INDEX ARE STRONGLY CORRELATED TO EACH OTHER AND ARE CHANGING IN THE SAME DIRECTION.(0.9145797)
* WOMEN ENTREPRENEURSHIP INDEX IS NEGATIVELY CORRELATED TO THE INFLATION RATE.(-0.45)


# TRENDS RELATED TO:

   *  LEVEL OF DEVELOPMENT
   *  EUROPEAN UNION MEMBERSHIP

In [ ]:
print("Count of Countries by level of development")
print(data["Level of development"].value_counts())

In [ ]:
title="Women Entrepreneurship Index vs Level of development"
fig = px.bar(data, x='Level of development', y='Women Entrepreneurship Index',color='Country',hover_data=["European Union Membership"]
             , height=600,width=900,template="simple_white",title=title)
fig.show()

In [ ]:
data["European Union Membership"].value_counts()

In [ ]:
title="Women Entrepreneurship Index vs Level of development"
fig = px.bar(data, x='European Union Membership', y='Women Entrepreneurship Index',color='Country',hover_data=["European Union Membership"]
             , height=600,width=900,template="simple_white",title=title)
fig.show()

# Developed Countries               
   1) Australia has the highest WEI of 74.8%                     
   2) Japan has the lowest WEI of 40.0%                          
   3) Average WEI is 59.36%                                      
   4) Median WEI is 58.5%                                         


# Developing Countries
   1) Uruguay has the highest WEI of 44.5%
   2) India has the lowest WEI of 25.3%
   3) Average WEI is 34.87%
   4) Median  WEI is 35.9%


# European Countires Membership
* **As per the data, all European Union Membership Countries are Developed**
* Denmark has the highest WEI of 69.7% 
* Greece has the lowest WEI of 43%
      
   
   
   
  

# Continent vs Women Entrpreneurship Index

In [ ]:

fig = px.bar(data, x='Continent', y='Women Entrepreneurship Index',color='Country',
              height=400,template="simple_white")
fig.show()

* average women entrepreneurship Index with respect to continents is:
* Europe: 57.04
* Africa: 27.9
* Asia: 40.99
* North America: 36.86
* South America: 36.15
* Ocenia:74.8

# WEI vs Inflation Rate

In [ ]:
fig = px.scatter(data,x="Women Entrepreneurship Index", y="Inflation rate",color="European Union Membership",
                 size="Women Entrepreneurship Index", template="simple_white",hover_data=['Country'],
                marginal_y="violin",marginal_x="box",trendline='ols', trendline_color_override='lightgreen')
fig.show()

# Analysis of scatter Plot
* inflation rate of European Union Members are between -2 and 1.
* WEI of European Union Members are relatively better than non-members.
* Among European Union members 
    * Greece has the lowest Inflation Rate of -1.7.
    * Austria has the highest Inflation Rate of 0.9
* Among Non-European Union members 
    * El Salvador has the lowest Inflation Rate of -2.25.
    * Argentina has the highest Inflation Rate of 26.5

# Analysis with respect to Female Labor Force Participation Rate 

In [ ]:
fig = px.scatter(data,x="Women Entrepreneurship Index", y="Female Labor Force Participation Rate",color="European Union Membership",
                 size="Women Entrepreneurship Index", template="simple_white",hover_data=['Country'],
                marginal_y="violin",marginal_x="box",trendline='ols', trendline_color_override='lightgreen')
fig.show()


 **European Union Members have higher Female Labor Force Participation Rate.**

In [ ]:
fig = px.sunburst(data, path=['Continent', 'Country'], values='Female Labor Force Participation Rate',
                  color='Female Labor Force Participation Rate', 
                  hover_data=['European Union Membership','Level of development','Women Entrepreneurship Index']
                 ,height=950,width=950)
fig.show()

* Arab Countries have lower Female Labor Force Participation Rate
* average Female Labor Force Participation Rates of Continents are:
   *   Europe: 64.64%
   *   Asia: 57.26%
   *   Ocenia: 66.8%
   *   Africa: 52.36%
   *   North America:55.24%
   *   South America:62.19%

# Task 1: NON-PARAMETRIC TEST

# Are European Union membership variable and development variable independent from each other? (Method Chi-Square Test and Crosstab)

In [ ]:
from scipy.stats import chi2_contingency
from scipy.stats import chi2
df = pd.crosstab(data['European Union Membership'], data['Level of development'])
df.head()

In [ ]:
expected_values_table=pd.DataFrame(chi2_contingency(df)[3],index=["Member","Not Member"],
    columns=["Developed","Developing"]).round(2)
expected_values_table

# *  **Null Hypothesis**:"Level of development" and "European Union Membership" are independent
# * **Alternate Hypothesis**:"Level of development" and "European Union Membership" are not independent

In [ ]:
chi=chi2_contingency(df)[0]
pval=chi2_contingency(df)[1]
dof=chi2_contingency(df)[2]
values_df=pd.DataFrame({"index":["chi square","pvalue","dof"],"values":[chi,pval,dof]}).round(2)
values_df

In [ ]:

print('p-value is: ', pval)
significance = 0.05
p = 1 - significance
critical_value = chi2.ppf(p, dof)
print('chi=%.6f, critical value=%.6f\n' % (chi, critical_value))
if chi > critical_value:
    print("""At %.2f level of significance, we reject the null hypotheses and accept H1. They are not independent.""" % (significance))
else:
    print("""At %.2f level of significance, we accept the null hypotheses. They are independent.""" % (significance))

# IF YOU FIND MY NOTEBOOK USEFUL, PLEASE LIKE IT!

![](http://media.tenor.com/images/37059d06865781137e128201f866ee6e/tenor.gif)